In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

players = pd.read_csv("players.csv")
player_play = pd.read_csv("player_play.csv")
plays = pd.read_csv("plays.csv")
y = plays[["expectedPointsAdded"]]

players = players[["position", "nflId", "displayName"]]

player_play = player_play[["playId", "nflId", "motionSinceLineset"]]

plays = plays[["playId", "yardsGained", "playAction", "offenseFormation", 
               "pff_passCoverage", "pff_manZone", "expectedPointsAdded"]]

player_info = pd.merge(players, player_play, on="nflId", how="inner")

combined_df = pd.merge(player_info, plays, on="playId", how="inner")

print(combined_df.head())



  position  nflId displayName  playId motionSinceLineset  yardsGained  \
0       QB  25511   Tom Brady     413              False           50   
1       QB  25511   Tom Brady     413              False            9   
2       QB  25511   Tom Brady     413              False            0   
3       QB  25511   Tom Brady     413              False           12   
4       QB  25511   Tom Brady     413              False            7   

   playAction offenseFormation pff_passCoverage pff_manZone  \
0       False          SHOTGUN          Cover-1         Man   
1        True          SHOTGUN    Cover-6 Right        Zone   
2       False            EMPTY     Cover 6-Left        Zone   
3       False       SINGLEBACK          Cover-3        Zone   
4       False          SHOTGUN    Cover-6 Right        Zone   

   expectedPointsAdded  
0             4.975587  
1             0.858816  
2            -0.675124  
3             0.821772  
4            -0.575678  


In [27]:
y = combined_df["expectedPointsAdded"]
X = combined_df.drop(columns=["expectedPointsAdded", "playId", "nflId", "displayName"])  # Exclude identifiers

X = pd.get_dummies(X, drop_first=True)

X = X.fillna(0)
y = y.fillna(0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.5317227472604728


In [28]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Ensure 'expectedPointsAdded' is in the combined DataFrame
plays = plays[["playId", "yardsGained", "playAction", "offenseFormation", 
               "pff_passCoverage", "pff_manZone", "expectedPointsAdded"]]
combined_df = pd.merge(player_info, plays, on="playId", how="inner")

# Define target variable and features
y = combined_df["expectedPointsAdded"]
X = combined_df.drop(columns=["expectedPointsAdded", "playId", "nflId", "displayName"])  # Exclude identifiers

# Handle categorical variables (e.g., one-hot encoding)
X = pd.get_dummies(X, drop_first=True)

# Handle missing values
X = X.fillna(0)
y = y.fillna(0)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Make predictions
y_pred = lr_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R^2 Score: {r2}")


Mean Squared Error: 0.7758855512802583
Mean Absolute Error: 0.5127899316589918
R^2 Score: 0.5928005793745719


In [30]:
print(X.dtypes)


const                                    float64
yardsGained                                int64
playAction                                  bool
position_CB                                 bool
position_DB                                 bool
position_DE                                 bool
position_DT                                 bool
position_FB                                 bool
position_FS                                 bool
position_G                                  bool
position_ILB                                bool
position_LB                                 bool
position_MLB                                bool
position_NT                                 bool
position_OLB                                bool
position_QB                                 bool
position_RB                                 bool
position_SS                                 bool
position_T                                  bool
position_TE                                 bool
position_WR         

In [31]:

# Convert boolean columns to integers (True -> 1, False -> 0)
X = X.astype(int)

# Handle missing values if any
X = X.fillna(0)  # Replace NaN with 0 in X
y = y.fillna(0)  # Replace NaN with 0 in y

# Add constant to the features for intercept
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X)
results = model.fit()

# Display the summary
print(results.summary())


                             OLS Regression Results                            
Dep. Variable:     expectedPointsAdded   R-squared:                       0.591
Model:                             OLS   Adj. R-squared:                  0.591
Method:                  Least Squares   F-statistic:                 5.692e+04
Date:                 Tue, 26 Nov 2024   Prob (F-statistic):               0.00
Time:                         15:56:25   Log-Likelihood:            -2.3046e+06
No. Observations:              1776097   AIC:                         4.609e+06
Df Residuals:                  1776051   BIC:                         4.610e+06
Df Model:                           45                                         
Covariance Type:             nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------